Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.


In [ ]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce
from collections import namedtuple
from tqdm import tqdm


In [ ]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets


# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

- `num_points = [100, 1_000, 5_000]`
- `num_sets = num_points`
- `density = [.3, .7]`


In [ ]:
# x = make_set_covering_problem(1000, 1000, 0.3)
# print("Element at row=42 and column=42:", x[42, 42])


# SOL


In [ ]:
State = namedtuple("State", ["taken", "cost", "heuristic"])


# solution check
def goal_check(state, problem, psize):
    return np.all(
        reduce(np.logical_or, [problem[i] for i in state.taken], np.zeros(psize))
    )


In [ ]:
# heuristic functions
def h1(taken, problem, psize):
    # number of spots covered by the union of the sets in state.taken
    return np.sum(reduce(np.logical_or, [problem[i] for i in taken], np.zeros(psize)))


# successor function
def successor1(state, problem, psize):
    return [
        State(state.taken + [i], state.cost + 1, h1(state.taken + [i], problem, psize))
        for i in range(psize)
        if i not in state.taken
    ]

def hill_climbing_1(init_state, problem, psize):
    current = init_state
    pb = tqdm()
    fit_calls = 0
    while True:
        neighbors = successor1(current, problem, psize)
        fit_calls += len(neighbors)
        next = max(neighbors, key=lambda state: state.heuristic)

        pb.update(1)
        if (
            goal_check(current, problem, psize)
            or next.heuristic < current.heuristic
            or len(current.taken) == psize
        ):
            return current, fit_calls

        current = next


In [ ]:
PROBLEM_SIZE = [100, 1000, 5000]
DENSITY = [0.3, 0.7]
INITIAL_STATE = State([], 0, 0)


def sparse_to_dense(sparse_matrix):
    dense_matrix = sparse_matrix.toarray()
    return dense_matrix


for p in PROBLEM_SIZE:
    for d in DENSITY:
        problem = sparse_to_dense(make_set_covering_problem(p, p, d))
        print("\nProblem size: ", p, "Density: ", d)

        solution, calls = hill_climbing_1(INITIAL_STATE, problem, p)

        print(" Solution: ", solution.taken)
        print(" Solution Lenght: ", solution.cost)
        print(" Fitness calls: ", calls)


Problem size: 100 Density: 0.3
7it [00:00, 282.91it/s]
Solution: [76, 0, 40, 3, 4, 17]
Solution Lenght: 6
Fitness calls: 679

Problem size: 100 Density: 0.7
4it [00:00, 297.12it/s]
Solution: [57, 7, 2]
Solution Lenght: 3
Fitness calls: 394

Problem size: 1000 Density: 0.3
11it [00:00, 26.89it/s]
Solution: [714, 404, 991, 572, 736, 951, 153, 212, 113, 6]
Solution Lenght: 10
Fitness calls: 10945

Problem size: 1000 Density: 0.7
5it [00:00, 41.03it/s]
Solution: [414, 105, 46, 15]
Solution Lenght: 4
Fitness calls: 4990

Problem size: 5000 Density: 0.3
14it [00:03, 4.05it/s]
Solution: [1075, 48, 3388, 2040, 803, 3699, 3296, 847, 4699, 3595, 1747, 852, 27]
Solution Lenght: 13
Fitness calls: 69909

Problem size: 5000 Density: 0.7
6it [00:01, 4.10it/s]
Solution: [309, 1976, 4590, 1639, 5]
Solution Lenght: 5
Fitness calls: 29985
